In [1]:
import pandas
df_english = pandas.read_csv("/home/toorajtaraz/Documents/university/IR/ClassProjects/MovieSummaries/plot_summaries.txt",
                             delimiter="\t",
                             names=["id", "text"])
df_english.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42303 entries, 0 to 42302
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      42303 non-null  int64 
 1   text    42303 non-null  object
dtypes: int64(1), object(1)
memory usage: 661.1+ KB


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/toorajtaraz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
texts_list = df_english["text"].tolist()
all_sentences = []
for text in texts_list:
    sentences = nltk.sent_tokenize(text)
    all_sentences.append(sentences)

all_sentences[0:10]

[["Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."],
 ['The nation of Panem consists of a wealthy Capitol and twelve poorer districts.',
  'As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games.',
  'The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth.',
  'In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12.',
  'Her older sister Katniss volunteers to take her place.',
  "Peeta Mellark, a baker's son who once gave Katniss bread when she was starving, is the other District 12 tribute.",
  'Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy.',
  'He warns them about the "Career" tributes who train intensively at

In [4]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



2023-04-11 19:12:58.271894: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [5]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
# Define the dataset of sentences to search
del dataset
dataset = []
for i, sentences in enumerate(all_sentences):
    if i < 10:
        for sentence in sentences:
            dataset.append(sentence)
    else:
        break
# Encode all the sentences in the dataset
dataset_embeddings = []
for sentence in dataset:
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.hidden_states[0][0].numpy()
    dataset_embeddings.append(embedding)

# Define a function to search for similar sentences
def search(query, k=5):
    # Encode the query sentence
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    query_embedding = outputs.hidden_states[0][0].numpy()
    
    # Compute the cosine similarity between the query embedding and each sentence embedding
    similarity_scores = []
    for sentence_embedding in dataset_embeddings:
        similarity_scores.append(cosine_similarity(query_embedding, sentence_embedding)[0][0])
    
    # Rank the sentences by similarity and return the top k
    top_k_indices = np.argsort(similarity_scores)[::-1][:k]
    top_k_sentences = [dataset[i] for i in top_k_indices]
    return top_k_sentences

# Test the search function with a sample query
query = "A man gives speech."
top_k = search(query)
print(f"The top 5 most similar sentences to '{query}' are:\n")
for i, sentence in enumerate(top_k):
    print(f"{i+1}. {sentence}") 

TypeError: 'NoneType' object is not subscriptable